<a href="https://colab.research.google.com/github/dawidkubicki/chatbot-tensorflow/blob/main/Chatbot_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to the GDrive

In [40]:
#from google.colab import drive
#drive.mount('/content/gdrive')

## Change root path

In [41]:
#root_path = '/content/gdrive/My Drive/Colab/'

import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "Colab/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'test_file.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/Colab/

An empty text file was created there. You can also run !pwd to confirm the current working directory.


## Download dataset



In [42]:
#!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
#!unzip cornell_movie_dialogs_corpus.zip

## Firstly import all necessaries libraries


In [43]:
import numpy as np
import tensorflow as tf
import re
import time

# Preprocessing

## Import dataset to the project

In [44]:
lines = open('dataset/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conversations = open('dataset/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

## Create a dictionary that maps each line and it's ID

In [45]:
id2line = {} #init dictionary
for line in lines:
  _line = line.split(' +++$+++ ')
  if len(_line) == 5:
    id2line[_line[0]] = _line[4]

# JUST TO CHECK DATA IN GOOGLE COLAB
# retrieve key/value pairs
#els = list(id2line.items()) # explicitly convert to a list, in case it's Python 3.x

# get first inserted element 
#print(els[0])



## Create a list of all of the conversations

In [46]:
conversations_ids = []
for conversation in conversations[:-1]:
  _conversation = conversation.split(" +++$+++ ")[-1][1:-1].replace("'","").replace(" ","")
  conversations_ids.append(_conversation.split(","))
#print(conversations_ids[0])
#print(conversations_ids[0])

## Getting separately the questions and the answers

In [47]:
questions = []
answers = []
for conversation in conversations_ids:
  for i in range(len(conversation) - 1):
    questions.append(id2line[conversation[i]])
    answers.append(id2line[conversation[i+1]])

#print(questions[0])
#print(answers[0])


### Cleaning text function define

In [48]:
def clean_text(text):
  #put all the text to lowercase
  text = text.lower()
  #removing aposthrophies with re library
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"what's", "what is", text)
  text = re.sub(r"where's", "where is", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'d", " whould", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
  return text

### Cleaning questions and answers

In [49]:
clean_questions = []
clean_answers = []

for question in questions:
  clean_questions.append(clean_text(question))

for answer in answers:
  clean_answers.append(clean_text(answer))  

### Creating a dictionary that maps each word to it's number of occurencies


In [50]:
word2count = {}

for question in clean_questions:
  for word in question.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1

for answer in clean_answers:
  for word in answer.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1      

### Creating two dictionaries that map the questions words and the answers words to a unique integer

In [51]:
threshold = 20
questionwords2int = {}
word_number = 0 
for word, count in word2count.items():
  if count > threshold:
    questionwords2int[word] = word_number
    word_number += 1

answerwords2int = {}
word_number = 0 
for word, count in word2count.items():
  if count > threshold:
    answerwords2int[word] = word_number
    word_number += 1

# print(answerwords2int.items())

### Adding the last tokens to these two dictionaries

In [52]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
  questionwords2int[token] = len(questionwords2int) + 1

for token in tokens:
  answerwords2int[token] = len(answerwords2int) + 1
#print(answerwords2int.items())  

### Create an inverse dictionary of the answerwords2int dictionary

In [53]:
answerint2word = {w_i: w for w, w_i in answerwords2int.items()}

### Adding the EOS token to the end of every answer

In [54]:
for i in range(len(clean_answers)):
  clean_answers[i] += ' <EOS>'

### Translating all the questions and the answers into integers and replacing all the words that were filtered out by <OUT>

In [55]:
questions_to_int = []
for question in clean_questions:
  ints = []
  for word in question.split():
    if word not in questionwords2int:
      ints.append(questionwords2int['<OUT>'])
    else:
      ints.append(questionwords2int[word])
  questions_to_int.append(ints)

print(questions_to_int[0])

answers_to_int = []
for answer in clean_answers:
  ints = []
  for word in answer.split():
    if word not in answerwords2int:
      ints.append(answerwords2int['<OUT>'])
    else:
      ints.append(answerwords2int[word])
  answers_to_int.append(ints)

[0, 1, 2, 3, 4, 8541, 8541, 5, 6, 8541, 7, 8, 9, 10, 8541, 11, 12, 13, 14, 15, 8541, 16]


### Sorting questions and answers by the length of questions (this will speed up the training)

In [56]:
sorted_clean_questions = []
sorted_clean_answers = []

for length in range(1,25+1):
  for i in enumerate(questions_to_int):
    if len(i[1]) == length:
      sorted_clean_questions.append(questions_to_int[i[0]])
      sorted_clean_answers.append(answers_to_int[i[0]])

print(sorted_clean_questions[0])
print(sorted_clean_answers[0])      

[47]
[15, 48, 25, 47, 18, 49, 50, 15, 51, 52, 45, 53, 8541, 54, 52, 55, 41, 56, 18, 57, 58, 59, 60, 61, 8540]


# Building seq2seq model

### Creating placeholders for the inputs and the targets

In [57]:
def model_inputs():
  inputs = tf.placeholder(tf.int32, [None, None], name = 'inputs')
  targets = tf.placeholder(tf.int32, [None, None], name = 'targets')
  lr = tf.placeholder(tf.float32, name = 'learning_rate')
  keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')

  return inputs, targets, lr, keep_prob

### Preprocessing the targets

In [58]:
# create batches and add <sos> token at each raw at batch

def preprocess_targets(targets, word2int, batch_size):
  left_side = tf.fill([batch_size], 1, word2int['<SOS>'])
  right_side = tf.strided_slice(targets, [0,0], [batch_size, -1],[1,1])
  preprocessed_targets = tf.concat([left_side, right_side], 1)

  return preprocessed_targets

### Creating the Encoder RNN Layer

In [60]:
def encoder_rnn_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
  #first create LSTM - object of basic lstm cell
  lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
  lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
  encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
  _, encoder_state = tf.nn_bidirectional_dynamic_rnn(cell_fw = encoder_cell, 
                                                     cell_bw = encoder_cell,
                                                     sequence_length = sequence_length,
                                                     inputs = rnn_inputs,
                                                     dtype = tf.float32) # this will build forward and backward RNNs

  return encoder_state